In [1]:
from selenium import webdriver
import pandas as pd
import os
from lxml import html
import numpy

In [9]:
import numpy

In [2]:
url = "https://www.masseyratings.com/nfl/ratings"

# create a new Firefox session
driver = webdriver.Chrome(executable_path='../chromedriver')
driver.get(url)

In [5]:
tree = html.fromstring(driver.page_source)

In [11]:
datatable = pd.DataFrame(
    dict(
        team_names=[x.text for x in tree.cssselect('.tan > a')],
        ratings=[float(x.text) for x in tree.cssselect('.sorted .detail')],
        offense=[
            float(x.text)
            for x in tree.cssselect('.frank:nth-child(6) .detail')
        ],
        defense=[
            float(x.text)
            for x in tree.csssealect('.frank:nth-child(7) .detail')
        ],
        hfa=[
            float(x.text)
            for x in tree.cssselect('.bodyrow .fShort:nth-child(8)')
        ],
    ))

In [12]:
datatable

,team_names,ratings,offense,defense,hfa
0,Kansas City,1.91,28.02,-1.05,2.47
1,LA Rams,1.77,26.52,1.14,1.83
2,Philadelphia,1.43,25.83,2.85,2.61
3,New Orleans,1.13,29.69,-2.84,2.32
4,Carolina,1.03,23.97,1.95,2.80
5,Tampa Bay,0.80,24.71,-0.97,2.56
6,Jacksonville,0.79,22.85,2.89,2.49
7,Atlanta,0.77,24.14,2.95,2.52
8,Baltimore,0.76,24.79,0.73,3.00
9,Pittsburgh,0.59,27.34,-2.04,2.37


In [13]:
driver.close()

In [10]:
sys.path.append('../')

In [11]:
import gspread
import pandas
from oauth2client.service_account import ServiceAccountCredentials

In [12]:
scope = [
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive'
]

credentials = ServiceAccountCredentials.from_json_keyfile_name(
    '/Users/dshurick/Downloads/dshurick-rdb-test-cb506c785735.json', scope)

spreadsheet = gspread.authorize(credentials)

worksheet = spreadsheet.open("NFL 2018 Expected Wins")

In [5]:
# scheduled_games = pandas.DataFrame(
#     worksheet.get_worksheet(0).get_all_records())[[
#         'week',
#         'away_team',
#         'home_team',
#         'home_adv',
#         'score_away',
#         'score_home',
#         'vegas_spread',
#     ]]

In [13]:
sagarin_ratings = pandas.DataFrame(
    worksheet.worksheet(title='sagarin').get_all_records())

scorex_ratings = pandas.DataFrame(
    worksheet.worksheet(title='scorex').get_all_records())

FiveThirtyEight_ratings = pandas.DataFrame(
    worksheet.worksheet(title='FiveThirtyEight').get_all_records())

massey_ratings = pandas.DataFrame(
    worksheet.worksheet(title='massey').get_all_records())

teaminfo = pandas.DataFrame(
    worksheet.worksheet(title='teaminfo').get_all_records())

In [9]:
import numpy

In [10]:
FiveThirtyEight_ratings['rating'] = FiveThirtyEight_ratings.rating.str.replace(
    ',', '').astype(numpy.int)

In [5]:
import gspread
import numpy
import pandas
from oauth2client.service_account import ServiceAccountCredentials

In [6]:
scope = [
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive'
]

credentials = ServiceAccountCredentials.from_json_keyfile_name(
    '/Users/dshurick/Downloads/dshurick-rdb-test-cb506c785735.json', scope)

spreadsheet = gspread.authorize(credentials)

worksheet = spreadsheet.open("NFL 2018 Expected Wins")

scheduled_games = pandas.DataFrame(
    worksheet.get_worksheet(0).get_all_records())[[
        'week',
        'away_team',
        'home_team',
        'home_adv',
        'score_away',
        'score_home',
        'vegas_spread',
    ]]

teaminfo = pandas.DataFrame(
    worksheet.worksheet(title='teaminfo').get_all_records())

In [10]:
teams = {
    x['team_name']: Team(
        team_id=x['team_id'],
        name=x['team_name'],
        conference=x['conference'],
        division=x['division'],
    )
    for x in teaminfo.to_dict('records')
}

In [1]:
%load_ext autoreload
%autoreload 2

In [360]:
from sportsball.nfl.nfl import Season, Team, Game

In [361]:
season_2018 = Season(
    year=2018,
    credfile='/Users/dshurick/Downloads/dshurick-rdb-test-cb506c785735.json',
    sheettitle='NFL 2018 Expected Wins',
)

In [362]:
season_2018.attach_sagarin_ratings()
season_2018.attach_538_ratings()
season_2018.attach_massey_ratings()
season_2018.attach_scorex_ratings()
season_2018.attach_vegas_ratings()

In [363]:
season_2018.set_game_probs('vegas')

In [364]:
season_2018.simulate()

{'Buffalo': {'wins': 1, 'losses': 15},
 'Miami': {'wins': 8, 'losses': 8},
 'N.Y. Jets': {'wins': 6, 'losses': 10},
 'New England': {'wins': 14, 'losses': 2},
 'Baltimore': {'wins': 9, 'losses': 7},
 'Cincinnati': {'wins': 9, 'losses': 7},
 'Cleveland': {'wins': 4, 'losses': 12},
 'Pittsburgh': {'wins': 4, 'losses': 12},
 'Denver': {'wins': 10, 'losses': 6},
 'Houston': {'wins': 7, 'losses': 9},
 'Indianapolis': {'wins': 6, 'losses': 10},
 'Jacksonville': {'wins': 14, 'losses': 2},
 'Kansas City': {'wins': 13, 'losses': 3},
 'L.A. Chargers': {'wins': 12, 'losses': 4},
 'Oakland': {'wins': 5, 'losses': 11},
 'Tennessee': {'wins': 8, 'losses': 8},
 'Dallas': {'wins': 10, 'losses': 6},
 'N.Y. Giants': {'wins': 5, 'losses': 11},
 'Philadelphia': {'wins': 8, 'losses': 8},
 'Washington': {'wins': 8, 'losses': 8},
 'Chicago': {'wins': 7, 'losses': 9},
 'Detroit': {'wins': 5, 'losses': 11},
 'Green Bay': {'wins': 11, 'losses': 5},
 'Minnesota': {'wins': 12, 'losses': 4},
 'Atlanta': {'wins': 1

In [367]:
sorted(list(season_2018.teams.values()))

[AFC East   | Buffalo         |  1 - 15,
 AFC East   | N.Y. Jets       |  6 - 10,
 AFC East   | Miami           |  8 -  8,
 AFC East   | New England     | 14 -  2,
 AFC North  | Cleveland       |  4 - 12,
 AFC North  | Pittsburgh      |  4 - 12,
 AFC North  | Baltimore       |  9 -  7,
 AFC North  | Cincinnati      |  9 -  7,
 AFC South  | Indianapolis    |  6 - 10,
 AFC South  | Houston         |  7 -  9,
 AFC South  | Tennessee       |  8 -  8,
 AFC South  | Jacksonville    | 14 -  2,
 AFC West   | Oakland         |  5 - 11,
 AFC West   | Denver          | 10 -  6,
 AFC West   | L.A. Chargers   | 12 -  4,
 AFC West   | Kansas City     | 13 -  3,
 NFC East   | N.Y. Giants     |  5 - 11,
 NFC East   | Philadelphia    |  8 -  8,
 NFC East   | Washington      |  8 -  8,
 NFC East   | Dallas          | 10 -  6,
 NFC North  | Detroit         |  5 - 11,
 NFC North  | Chicago         |  7 -  9,
 NFC North  | Green Bay       | 11 -  5,
 NFC North  | Minnesota       | 12 -  4,
 NFC South  | Ta

In [314]:
sorted(
    [{
        'name': team.name,
        'division': team.division,
        'conference': team.conference,
        'wins': team.wins,
        'losses': team.losses,
    } for team in season_2018.teams.values()],
    key=lambda x: -x['wins'])

[{'name': 'Buffalo',
  'division': 'AFC East',
  'conference': 'AFC',
  'wins': 1,
  'losses': 15},
 {'name': 'Indianapolis',
  'division': 'AFC South',
  'conference': 'AFC',
  'wins': 3,
  'losses': 13},
 {'name': 'Denver',
  'division': 'AFC West',
  'conference': 'AFC',
  'wins': 4,
  'losses': 12},
 {'name': 'Arizona',
  'division': 'NFC West',
  'conference': 'NFC',
  'wins': 5,
  'losses': 11},
 {'name': 'Cleveland',
  'division': 'AFC North',
  'conference': 'AFC',
  'wins': 6,
  'losses': 10},
 {'name': 'Oakland',
  'division': 'AFC West',
  'conference': 'AFC',
  'wins': 6,
  'losses': 10},
 {'name': 'Dallas',
  'division': 'NFC East',
  'conference': 'NFC',
  'wins': 6,
  'losses': 10},
 {'name': 'Tampa Bay',
  'division': 'NFC South',
  'conference': 'NFC',
  'wins': 6,
  'losses': 10},
 {'name': 'Seattle',
  'division': 'NFC West',
  'conference': 'NFC',
  'wins': 6,
  'losses': 10},
 {'name': 'Pittsburgh',
  'division': 'AFC North',
  'conference': 'AFC',
  'wins': 7,
  '

In [308]:
df['pct'] = df.wins / (df.wins + df.losses)

In [312]:
df.sort_values(
    ['division', 'pct'], ascending=[True, False])[[
        'name',
        'conference',
        'division',
        'wins',
        'losses',
        'pct',
    ]]

,name,conference,division,wins,losses,pct
1,Miami,AFC,AFC East,10,6,0.6250
2,N.Y. Jets,AFC,AFC East,10,6,0.6250
3,New England,AFC,AFC East,10,6,0.6250
0,Buffalo,AFC,AFC East,1,15,0.0625
4,Baltimore,AFC,AFC North,11,5,0.6875
5,Cincinnati,AFC,AFC North,8,8,0.5000
7,Pittsburgh,AFC,AFC North,7,9,0.4375
6,Cleveland,AFC,AFC North,6,10,0.3750
11,Jacksonville,AFC,AFC South,13,3,0.8125
15,Tennessee,AFC,AFC South,9,7,0.5625


In [1]:
import pandas
import sklearn

from sklearn_pandas import DataFrameMapper, cross_val_score
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, log_loss

In [168]:
from sklearn.model_selection import GroupKFold

In [148]:
from sklearn.linear_model import SGDClassifier, LogisticRegression

In [8]:
dtf = pandas.read_csv('./../data/processed/nfl_data.csv')
dtf = dtf.loc[(dtf.home_win == 0) | (dtf.home_win == 1)]

In [239]:
dtf

,season,five38_home_rating,five38_away_rating,massey_home_rating,massey_away_rating,sagarin_home_rating,sagarin_away_rating,home_win,gamenum,home_adv
0,2017,1434,1559,24.83,22.80,17.12,20.50,1.0,1,1
1,2017,1529,1600,21.28,18.56,23.01,21.80,1.0,2,1
2,2017,1476,1497,21.00,24.19,20.23,18.40,1.0,3,1
3,2017,1498,1480,21.69,11.05,24.08,17.84,1.0,4,1
4,2017,1623,1489,32.83,15.50,27.78,19.01,1.0,5,1
5,2017,1489,1387,29.52,23.75,24.02,11.43,0.0,6,1
6,2017,1455,1514,17.59,25.61,19.77,21.57,0.0,7,1
7,2017,1598,1436,31.03,18.36,27.66,19.74,1.0,8,1
8,2017,1282,1460,20.29,22.22,12.33,14.98,0.0,9,1
9,2017,1608,1477,24.66,21.94,26.23,20.71,0.0,10,1


In [144]:
pipe = sklearn.pipeline.Pipeline([
    ('featurize',
     DataFrameMapper(
         [
             (['five38_home_rating'], StandardScaler()),
             (['five38_away_rating'], StandardScaler()),
             (['massey_home_rating'], StandardScaler()),
             (['massey_away_rating'], StandardScaler()),
             (['sagarin_home_rating'], StandardScaler()),
             (['sagarin_away_rating'], StandardScaler()),
         ],
         sparse=True)),
    ('lm',
     SGDClassifier(
         alpha=0.08,
         loss='log',
         fit_intercept=False,
         penalty='l2',
         max_iter=1000,
         tol=1e-3,
     )),
])

In [230]:
pipe = sklearn.pipeline.Pipeline([
    ('featurize',
     DataFrameMapper(
         [
             ([
                 'five38_away_rating',
                 'five38_home_rating',
                 'massey_away_rating',
                 'massey_home_rating',
                 'sagarin_away_rating',
                 'sagarin_home_rating',
             ], StandardScaler()),
         ],
         sparse=True)),
    ('lm', LogisticRegression(C=0.4)),
])

In [232]:
x = pipe.named_steps['featurize']

In [236]:
x.features[0][1].scale_

array([97.86976773, 97.86976773,  4.51784368,  4.51784368,  4.5378788 ,
        4.5378788 ])

In [237]:
x.features[0][1].mean_

array([1513.71610169, 1513.71610169,   23.00144068,   23.00144068,
         21.25025424,   21.25025424])

In [ ]:
cv = GroupKFold(n_splits=20)

In [206]:
xxx = cross_val_score(
    pipe,
    X=dtf,
    y=dtf.home_win,
    groups=dtf.gamenum,
    scoring=make_scorer(
        log_loss,
        greater_is_better=False,
        needs_proba=True,
    ),
    cv=cv.split(X=dtf, groups=dtf.gamenum),
    n_jobs=-1,
)

In [207]:
xxx.mean()

-0.5844360835663382

In [ ]:
clf = GridSearchCV(svc, parameters, scor)

In [231]:
pipe.fit(X=dtf, y=dtf.home_win)

Pipeline(memory=None,
     steps=[('featurize', DataFrameMapper(default=False, df_out=False,
        features=[(['five38_away_rating', 'five38_home_rating', 'massey_away_rating', 'massey_home_rating', 'sagarin_away_rating', 'sagarin_home_rating'], StandardScaler(copy=True, with_mean=True, with_std=True))],
        input_df=Fa...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [220]:
lm = pipe.named_steps['lm']

In [235]:
lm.coef_

array([[-0.48811602,  0.48811602,  0.83260993, -0.83260993,  0.17701139,
        -0.17701139]])

In [238]:
0.48811602/97.86976773

0.004987403478330498